In [102]:
import pickle
from pclpy import pcl as pcl2
import pcl
import pcl.pcl_visualization
import time
import numpy as np
import cv2
import matplotlib.pyplot as plt
from open3d import *
import pptk


In [35]:
def depthFrametoPC(depth, rgb, cameraIntrinsics, poseMat ):
    #starttime = time.time()
    #depth = deviceData['depth']
    #infrared = deviceData['infrared']
    #rgb = cv2.cvtColor(np.asanyarray(infrared),cv2.COLOR_GRAY2RGB)
    #cameraIntrinsics = deviceData['intrinsics']
    #poseMat = deviceData['poseMat']
    [height,width] = np.array(depth.shape)
    nx = np.linspace(0, width-1, width)
    ny = np.linspace(0, height-1, height)
    u, v = np.meshgrid(nx, ny)
    x = (u.flatten() - cameraIntrinsics['ppx'])/cameraIntrinsics['fx']
    y = (v.flatten() - cameraIntrinsics['ppy'])/cameraIntrinsics['fy']
    z = depth.flatten() / 1000
    x = np.multiply(x,z)
    y = np.multiply(y,z)
    ##time2 = time.time()
    #x = x[np.nonzero(z)]
    #y = y[np.nonzero(z)]
    #z = z[np.nonzero(z)]

    rgbB = rgb[:,:,0].flatten().astype(int)
    rgbG = rgb[:,:,1].flatten().astype(int)
    rgbR = rgb[:,:,2].flatten().astype(int)
    rgbPC = rgbR<<16|rgbG<<8|rgbB
    rgbThres = 10<<16|10<<8|10
    ##time3=time.time()
    points = np.asanyarray([x,y,z])

    n = points.shape[1] 
    points_ = np.vstack((points, np.ones((1,n))))
    points_trans_ = np.matmul(poseMat, points_)
    points_transformed = np.true_divide(points_trans_[:3,:], points_trans_[[-1], :])
    ##time4=time.time()
    allPoints = np.asanyarray([points_transformed[0,:],points_transformed[1,:],points_transformed[2,:],rgbB/256, rgbG/256, rgbR/256]).T
    #allPoints = allPoints[np.where(rgbPC>=rgbThres),:]
    ##times = np.array([time1,time2,time3,time4])-starttime
    ##print(times)
    return allPoints

In [3]:
#obtain background frames for background subtraction
filename = 'loco15static.pickle'
file = open(filename, 'rb')
#dataRead = []
#cloud = pcl.PointCloud_PointXYZRGBA()
#visual = pcl.pcl_visualization.CloudViewing()
#time.sleep(2)
#colorFrames = [np.empty((848,480,3)) for i in range(0,6)]
bgFrames = {}
i = 1
while 1:
    try:
        frame = pickle.load(file) #this loads each frame in one at a time, this is where we can process each frame with the transformation matrix
        cloud = pcl.PointCloud_PointXYZRGBA()
        #cloud2 = pcl.PointCloud_PointXYZRGBA()
        allPoints = np.empty((0,4))
        #for (serial, [poseMat, rmsdValue]) in self.devicesTransformation.items():
        for camera,deviceData in frame.items():
            #depthFrame = deviceData['depth']
            colorFrame = deviceData['infrared']
            if i == 1:
                bgFrames[camera] = colorFrame
            else:
                bgFrames[camera] = cv2.addWeighted(bgFrames[camera],0.5,colorFrame,0.5,0)
            #cameraIntrinsics = deviceData['intrinsics']
            #poseMat = deviceData['poseMat']
            #points = depthFrametoPC(depthFrame, colorFrame, cameraIntrinsics, poseMat)
            #allPoints = np.append(allPoints, points,axis=0)
        #cloud.from_array(allPoints.astype('float32'))

        #visual.ShowColorACloud(cloud)
        print(i)
        i+=1
    except EOFError:
        break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
cv2.imshow('color', bgFrames['822512060853'])
cv2.waitKey(1)

In [4]:
def open3d_to_pcl(pointCloud):
    pclCloud = pcl.PointCloud_PointXYZRGBA()
    points = np.asarray(pointCloud.points)
    colors = np.asarray(pointCloud.colors)
    colors255 = np.asarray([[r*255,g*255,b*255] for r,g,b in colors])
    rgbPC = np.asarray([r.astype('int')<<16|g.astype('int')<<8|b.astype('int') for r,g,b in colors255]).reshape((-1, 1)) 
    pclFormat = np.hstack((points, rgbPC))
    rgbThres = 200<<16|200<<8|200
    pclFormat[np.where(pclFormat[:,3]>=rgbThres),:][0,:,:]
    pclCloud.from_array(pclFormat.astype('float32'))
    return pclCloud
    
def pcl_to_open3d(pointCloud):
    #pclCloud = pcl.PointCloud_PointXYZRGBA()
    pcd = PointCloud()
    pclPC = np.asarray(pclPC)
    points = pclPC[:,0:3]
    colors = pclPC[:,3]
    colors = [[r>>16,g>>8,b] for r,g,b in colors]
    colors255 = np.asarray([[r*255,g*255,b*255] for r,g,b in colors])
    rgbPC = np.asarray([r.astype('int')<<16|g.astype('int')<<8|b.astype('int') for r,g,b in colors255]).reshape((-1, 1)) 
    pclFormat = np.hstack((points, rgbPC))
    pclCloud.from_array(pclFormat.astype('float32'))
    return pclCloud
    
    

In [193]:
filename = 'loco15walk1.pickle'
file = open(filename, 'rb')
#dataRead = []
#cloud = pcl.PointCloud_PointXYZRGBA()
#visual = pcl.pcl_visualization.CloudViewing()
#visual.setBackgroundColor (1.0, 0.5, 1.0)
#time.sleep(6)
detector = cv2.SimpleBlobDetector_create()
i = 1
#interest = [93,97,100,105]
while 1:
    try:
        frame = pickle.load(file) #this loads each frame in one at a time, this is where we can process each frame with the transformation matrix
        #cloud = pcl.PointCloud_PointXYZRGBA()
        #if i in interest:
        pcd = PointCloud()
        mpcd = PointCloud()
        allPoints = np.empty((0,6))
        markerPoints = np.empty((0,6))
        #for (serial, [poseMat, rmsdValue]) in self.devicesTransformation.items():
        for camera,deviceData in frame.items():
            depthFrame = deviceData['depth']
            infraredFrame = deviceData['infrared']
            inff = cv2.cvtColor(np.asanyarray(infraredFrame), cv2.COLOR_GRAY2RGB)
            inff[:,:,0][inff[:,:,0]<215] = 0
            inff[:,:,1][inff[:,:,1]<215] = 255
            inff[:,:,2][inff[:,:,2]<215] = 112
            inff[:,:,0][inff[:,:,0]>215] = 255
            inff[:,:,1][inff[:,:,1]>215] = 100
            inff[:,:,2][inff[:,:,2]>215] = 0
            #cv2.imshow('color',cv2.cvtColor(np.asanyarray(colorFrame),cv2.COLOR_GRAY2RGB))
            #cv2.waitKey(1)
            """
            infraredFrame = cv2.absdiff(bgFrames[camera],deviceData['infrared'])
            lower = np.array([220])
            upper = np.array([256])
            mask = cv2.inRange(infraredFrame, lower, upper)
            markerFilt = cv2.bitwise_and(infraredFrame,infraredFrame, mask= mask)
            maskInv = cv2.bitwise_not(mask)
            newColor = np.zeros((480,848, 3), np.uint8)
            newColor[:] = (255, 102, 0)
            bk = cv2.bitwise_and(newColor, newColor, mask=maskInv)
            markerFilt = cv2.cvtColor(np.asanyarray(markerFilt),cv2.COLOR_GRAY2RGB)
            newImg = cv2.bitwise_or(markerFilt, bk)
            depthFilt = cv2.bitwise_and(depthFrame,depthFrame, mask= mask)
            #_, difference = cv2.threshold(colorFrame, 25, 255, cv2.THRESH_BINARY)
            cv2.imshow('color',newImg)
            #cv2.imshow('color', np.hstack((colorFrame, deviceData['infrared'],bgFrames[camera])))
            cv2.waitKey(1)
            
            newAr
            """
            cameraIntrinsics = deviceData['intrinsics']
            poseMat = deviceData['poseMat']
            points = depthFrametoPC(depthFrame, inff, cameraIntrinsics, poseMat)
            ##markerPoint = depthFrametoPC(depthFilt, newImg, cameraIntrinsics, poseMat)
            #pclcolors = np.asanyarray(points[:,3]*256).astype('int')<<16|np.asanyarray(points[:,4]*256).astype('int')<<8|np.asanyarray(points[:,5]*256).astype('int')
            #rgbThres = 215<<16|215<<8|215
            markerPoint = points[points[:,3]>0]
            allPoints = np.append(allPoints, points,axis=0)
            markerPoints = np.append(markerPoints, markerPoint, axis=0)
        #cloud.from_array(allPoints.astype('float32'))
        pcd.points=Vector3dVector(allPoints[:,0:3])
        pcd.colors=Vector3dVector(allPoints[:,3:6])
        mpcd.points=Vector3dVector(markerPoints[:,0:3])
        mpcd.colors=Vector3dVector(markerPoints[:,3:6])
        #mpcl = open3d_to_pcl(mpcd)
        voxel_down_pcd = voxel_down_sample(pcd, voxel_size = 0.005)
        cl,ind = statistical_outlier_removal(voxel_down_pcd, nb_neighbors=40, std_ratio=2.0)
        pclPC = open3d_to_pcl(cl)
        for j in range(0,3):
            seg = pclPC.make_segmenter_normals(ksearch=50)
            seg.set_optimize_coefficients(True)
            seg.set_model_type(pcl.SACMODEL_NORMAL_PLANE)
            seg.set_method_type(pcl.SAC_RANSAC)
            seg.set_distance_threshold(0.005)
            seg.set_normal_distance_weight(0.003)
            seg.set_max_iterations(100)
            indices, coefficients = seg.segment()
            pclPC = pclPC.extract(indices, negative=True)
        if indices != 0:
            #clFilt = pclPC.extract(indices, negative=True)
            #finalPC = np.asarray(pclPC)
            #mPoints = np.asarray(mpcl)
            #finalPCL = pcl.PointCloud_PointXYZRGBA()
            #finalPCL.from_array(np.append(finalPC,mPoints, axis=0).astype('float32'))
            #visual.ShowColorACloud(clFilt)
            v = pptk.viewer(np.vstack((np.asarray(pclPC)[:,0:3], markerPoints[:,0:3])))
            v.set(lookat = [0.15542246,0.05657531,-0.15577665],
                  theta=-3.06796193, 
                  phi=2.87161231, 
                  r =0.62589908,
                  point_size=0.0015,
                  show_info=False, 
                  show_axis=False,
                  show_grid=0)
            colors = np.hstack((np.full((len(np.asarray(pclPC)),1),1),np.full((len(np.asarray(pclPC)),1),1),np.full((len(np.asarray(pclPC)),1),1)))
            v.attributes(np.vstack((colors, markerPoints[:,3:6])))
            v.capture("C:/Users/abhis/Dropbox/gradResearch/Projects/EVABootFit/RealSense_capture/clpics/"+str(i)+"screenshot.png")
            #v.close()
            #v.set(show_grid=0)
            #time.sleep(6)
            #print(v.get('lookat'), v.get('theta'),v.get('phi'),v.get('r'))
                #if i in interest:
                    #pcl.save(clFilt, str(i),format='ply')
            #draw_geometries([cl])
        else:
            visual.ShowColorACloud(pclPC)

        print(i, len(indices), len(markerPoints), len(allPoints))
        i+=1
    except EOFError:
        break
file.close()

1 4937 1822 2442240
2 2071 1799 2442240
3 4363 1782 2442240
4 4275 1796 2442240
5 5243 1796 2442240
6 3221 1867 2442240
7 4926 1903 2442240
8 2775 1846 2442240
9 4328 1818 2442240
10 3299 1846 2442240
11 1936 1761 2442240
12 3247 1701 2442240
13 3460 1744 2442240
14 3338 1815 2442240
15 3868 1674 2442240
16 5434 1715 2442240
17 3030 1758 2442240
18 2379 1766 2442240
19 4219 1849 2442240
20 1617 1666 2442240
21 2262 1780 2442240
22 2357 1643 2442240
23 2363 1579 2442240
24 2213 1609 2442240
25 3140 1546 2442240
26 5487 1515 2442240
27 2948 1407 2442240
28 2833 1491 2442240
29 2667 1524 2442240
30 2371 1502 2442240
31 3579 1466 2442240
32 3498 1386 2442240
33 3654 1270 2442240
34 2750 1146 2442240
35 3526 1134 2442240
36 1968 1031 2442240
37 2209 966 2442240
38 2451 932 2442240
39 2530 893 2442240
40 4071 863 2442240
41 2811 752 2442240
42 4447 717 2442240
43 2823 673 2442240
44 3223 643 2442240
45 3928 621 2442240
46 3892 581 2442240
47 4024 619 2442240
48 3338 665 2442240
49 5139 674 2

In [25]:
p = np.asanyarray(allPoints[:,3]*256).astype('int')<<16|np.asanyarray(allPoints[:,4]*256).astype('int')<<8|np.asanyarray(allPoints[:,5]*256).astype('int')
rgbThres = 10<<16|10<<8|10
allPoints[np.where(p>=rgbThres)]

array([[ 0.08518825,  0.18596674, -0.0905021 ,  0.1640625 ,  0.1640625 ,
         0.1640625 ],
       [ 0.08567917,  0.18437481, -0.09096862,  0.17578125,  0.17578125,
         0.17578125],
       [ 0.08617106,  0.18278593, -0.09143516,  0.18359375,  0.18359375,
         0.18359375],
       ...,
       [ 0.56478601, -0.03154772,  0.00694166,  0.05078125,  0.05078125,
         0.05078125],
       [ 0.56519883, -0.03233387,  0.00695803,  0.04296875,  0.04296875,
         0.04296875],
       [ 0.56561164, -0.03312003,  0.0069744 ,  0.04296875,  0.04296875,
         0.04296875]])

In [49]:
cv2.imshow('mask2', inff)
cv2.waitKey(1)

-1

In [60]:
visual.ShowColorACloud(finalPCL)

In [172]:
np.asarray(pclPC)[:,0:3]

array([[ 8.2655571e-02,  1.7441772e-01, -6.5673240e-02],
       [ 3.5670052e-41,  0.0000000e+00,  0.0000000e+00],
       [ 2.7768365e-01, -2.5558037e-01,  2.5120773e-03],
       ...,
       [ 3.5670052e-41,  0.0000000e+00,  0.0000000e+00],
       [ 1.4287831e-01,  1.1959000e-01, -2.5106683e-01],
       [ 3.5670052e-41,  0.0000000e+00,  0.0000000e+00]], dtype=float32)

In [48]:
inff = cv2.cvtColor(np.asanyarray(infraredFrame), cv2.COLOR_GRAY2RGB)
inff[:,:,0][inff[:,:,0]<215] = 255
inff[:,:,1][inff[:,:,1]<215] = 102
inff[:,:,2][inff[:,:,2]<215] = 0

In [96]:
pointCloud = cl
points = np.asarray(pointCloud.points)
colors = np.asarray(pointCloud.colors)
colors255 = np.asarray([[r*255,g*255,b*255] for r,g,b in colors])
rgbPC = np.asarray([r.astype('int')<<16|g.astype('int')<<8|b.astype('int') for r,g,b in colors255]).reshape((-1, 1)) 
pclFormat = np.hstack((points, rgbPC))
pclFormat

array([[ 1.96333892e-01,  1.72829803e-01,  5.16622892e-03,
         1.71139100e+06],
       [ 2.48649545e-01, -6.97542828e-02, -3.68242425e-01,
         7.50247500e+06],
       [ 6.36214069e-01, -1.81868019e-01,  5.31033166e-03,
         7.22438000e+05],
       ...,
       [-3.72103548e-01, -8.55169513e-02,  5.61818514e-03,
         4.53791300e+06],
       [-3.21660779e-01, -1.16328530e-01,  1.02665957e-02,
         4.34233500e+06],
       [-3.41121281e-01, -1.05749184e-01,  6.77716879e-03,
         8.53250000e+05]])

In [114]:
seg = pclPC.make_segmenter_normals(ksearch=50)

In [161]:
colors = np.hstack((np.full((len(np.asarray(pclPC)),1),1),np.full((len(np.asarray(pclPC)),1),1),np.full((len(np.asarray(pclPC)),1),1)))
np.vstack((colors, markerPoints[:,3:6]))

array([[1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        ],
       ...,
       [0.99609375, 0.390625  , 0.        ],
       [0.99609375, 0.390625  , 0.        ],
       [0.99609375, 0.390625  , 0.        ]])

In [160]:
markerPoints[:,3:6].shape

(1782, 3)

In [128]:
pcd = PointCloud()
pclPC = np.asarray(pclPC)
points = pclPC[:,0:3]
colors = pclPC[:,3].astype('int')
colors = [[r>>16,g>>8,b] for r,g,b in colors]

TypeError: 'numpy.int32' object is not iterable

In [ ]:
file.close()